In [10]:
import pandas as pd
import numpy as np
import os
from os.path import join as oj
from pose_analysis_base import *

In [11]:
root_folder = r'/Users/yang/Documents/Wilbrecht_Lab/data4analysis/processed_tracks'

posemat = PoseMat(root_folder)
combined_df = posemat.get_combined_df()

Loading combined DataFrame...


In [12]:
print(combined_df.shape)
combined_df.head()

(1643207, 39)


,Unnamed: 0,Head x,Head y,Neck x,Neck y,Torso x,Torso y,Tailhead x,Tailhead y,warped Head x,...,Head velocity,Neck velocity,Torso velocity,Tailhead velocity,Head acceleration,Neck acceleration,Torso acceleration,Tailhead acceleration,animal,session
0,0,192.796951,28.332069,195.778046,23.389879,199.605362,11.785733,203.450699,6.817648,273.703649,...,0.000000,0.000000,0.000000,0.000000,6649.885821,4646.693069,4412.589343,590.246091,RRM028,Day141
1,1,192.110931,35.688915,195.196640,28.520031,199.130478,16.665558,203.038986,7.328142,274.235963,...,221.662861,154.889769,147.086311,19.674870,312.597565,2818.161807,3099.677421,4895.582548,RRM028,Day141
2,2,191.869446,43.421238,192.651031,36.414017,196.267807,24.506279,199.963287,12.590609,274.606919,...,232.082780,248.828496,250.408892,182.860955,-2067.092682,-755.839578,-671.529336,1497.167579,RRM028,Day141
3,3,191.725861,48.858665,192.430756,43.865223,195.759293,32.090069,199.434418,20.331448,275.048460,...,163.179690,223.633843,228.024581,232.766541,2402.980104,552.449666,-58.973204,-460.739103,RRM028,Day141
4,4,191.152802,56.947693,191.896271,51.915794,195.551804,39.622505,199.436478,27.578400,276.035795,...,243.279027,242.048832,226.058807,217.408570,-1.766716,-338.857171,218.826253,883.502202,RRM028,Day141


In [13]:
def compare_decisions(df):
    """

    """
    current_trial_num = np.nan
    current_bonsai_decision = np.nan
    current_decision = np.nan
    bonsai_decision = np.nan
    decision = np.nan
    
    bonsai_decision_list = []
    decision_list = []
    animal_list = []
    session_list = []
    trial_list = []

    for index, row in df.iterrows():
        bonsai_decision = row['label']
        decision = row['decision']
        trial_num = row['trial']
        animal = row['animal']
        session = row['session']
        
        if trial_num != current_trial_num: # End of current trial and start of a new trial 
            if current_bonsai_decision == 'collection':
                current_bonsai_decision = 'ACC'
            if current_decision != current_bonsai_decision:
                bonsai_decision_list.append(current_bonsai_decision)
                decision_list.append(current_decision)
                animal_list.append(animal)
                session_list.append(session)
                trial_list.append(current_trial_num)

            current_bonsai_decision = np.nan    
            current_decision = np.nan
            current_trial_num = trial_num 
              
        if not pd.isna(decision):
            current_decision = decision    
        if not pd.isna(bonsai_decision):
            current_bonsai_decision = bonsai_decision
    
    result_df = pd.DataFrame({
        'bonsai decision': bonsai_decision_list,
        'decision': decision_list,
        'animal': animal_list,
        'session': session_list,
        'trial': trial_list
    })
    
    return result_df

In [14]:
different_trials = compare_decisions(combined_df)

In [15]:
#different_trials.to_csv(oj(root_folder, 'different_trials.csv'), index=False)
print(different_trials.shape)
different_trials.head()

(2248, 5)


,bonsai decision,decision,animal,session,trial
0,NaN,NaN,RRM028,Day141,NaN
1,quit,REJ,RRM028,Day141,20.0
2,quit,REJ,RRM028,Day141,66.0
3,quit,REJ,RRM028,Day141,85.0
4,quit,REJ,RRM028,Day141,97.0


In [16]:
ACC_REJ_trials = different_trials[
    ((different_trials['bonsai decision'] == 'ACC') & (different_trials['decision'] == 'REJ')) |
    ((different_trials['bonsai decision'] == 'REJ') & (different_trials['decision'] == 'ACC'))
]

quit_REJ_trials = different_trials[
    (different_trials['bonsai decision'] == 'quit') & (different_trials['decision'] == 'REJ')
]

print(len(ACC_REJ_trials))
print(len(quit_REJ_trials))

44
1903


In [17]:
a = different_trials[
    different_trials['bonsai decision'] == np.nan
]
a

,bonsai decision,decision,animal,session,trial


In [18]:
# Get unique combinations of decisions that exist in the DataFrame
existing_combinations = different_trials[['bonsai decision', 'decision']].drop_duplicates()

# Convert to list of tuples
existing_combinations_list = [tuple(x) for x in existing_combinations.to_records(index=False)]

# Display the combinations
print(existing_combinations_list)

[(nan, nan), ('quit', 'REJ'), ('T_Entry', 'REJ'), ('T_Entry', 'quit'), ('T_Entry', nan), ('REJ', 'quit'), ('outcome', 'quit'), ('tone_onset', 'ACC'), ('quit', 'ACC'), ('ACC', 'REJ'), ('ACC', 'T-Entry'), ('T_Entry', 'T-Entry'), ('ACC', nan), ('outcome', 'ACC'), ('T_Entry', 'ACC'), (nan, 'quit'), ('tone_onset', 'quit'), ('outcome', 'REJ'), ('ACC', 'quit'), ('tone_onset', 'REJ')]
